In [3]:
%matplotlib inline

import keen
from keen.client import KeenClient
import arrow

KEEN_PROJECT_ID = "572dfdae3831443195b2f30c"
KEEN_READ_KEY = "5de7f166da2e36f6c8617347a7a729cfda6d5413db8d88d7f696b61ddaa4fe1e5cdb7d019de9bb0ac846d91e83cdac01e973585d0fba43fadf92f06a695558b890665da824a0cf6a946ac09f5746c9102d228a1165323fdd0c52c92b80e78eca"

client = KeenClient(
    project_id=KEEN_PROJECT_ID,
    read_key=KEEN_READ_KEY
)


In [12]:
client.count("earthquakes",
    timeframe={
        "start": arrow.get(2016, 9, 19).format(),
        "end": arrow.get(2016, 11, 2).format()
    },
    interval="daily"
)

[{'timeframe': {'end': '2016-09-20T00:00:00.000Z',
   'start': '2016-09-19T00:00:00.000Z'},
  'value': 77},
 {'timeframe': {'end': '2016-09-21T00:00:00.000Z',
   'start': '2016-09-20T00:00:00.000Z'},
  'value': 284},
 {'timeframe': {'end': '2016-09-22T00:00:00.000Z',
   'start': '2016-09-21T00:00:00.000Z'},
  'value': 286},
 {'timeframe': {'end': '2016-09-23T00:00:00.000Z',
   'start': '2016-09-22T00:00:00.000Z'},
  'value': 262},
 {'timeframe': {'end': '2016-09-24T00:00:00.000Z',
   'start': '2016-09-23T00:00:00.000Z'},
  'value': 300},
 {'timeframe': {'end': '2016-09-25T00:00:00.000Z',
   'start': '2016-09-24T00:00:00.000Z'},
  'value': 287},
 {'timeframe': {'end': '2016-09-26T00:00:00.000Z',
   'start': '2016-09-25T00:00:00.000Z'},
  'value': 367},
 {'timeframe': {'end': '2016-09-27T00:00:00.000Z',
   'start': '2016-09-26T00:00:00.000Z'},
  'value': 473},
 {'timeframe': {'end': '2016-09-28T00:00:00.000Z',
   'start': '2016-09-27T00:00:00.000Z'},
  'value': 509},
 {'timeframe': {'end